In [1]:
import pandas as pd

In [2]:
url = 'https://bit.ly/3pmchka'
df = pd.read_csv(url)

print(df.head())

         Date       Open       High        Low      Close    Volume  Adj Close
0  2012-01-03  59.970001  61.060001  59.869999  60.330002  12668800  52.619235
1  2012-01-04  60.209999  60.349998  59.470001  59.709999   9593300  52.078475
2  2012-01-05  59.349998  59.619999  58.369999  59.419998  12768200  51.825539
3  2012-01-06  59.419998  59.450001  58.869999  59.000000   8069400  51.459220
4  2012-01-09  59.029999  59.549999  58.919998  59.180000   6679300  51.616215


In [3]:
# Installing pyspark
# ---
#
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=bc1bdef0cda9698171ad4f035792fa3a454ebc51ea3086719577f37efbeabc50
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
# Next, we run a local spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [6]:
#Importing the required libraries

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

# Start a Spark session
spark = SparkSession.builder.appName('SafaricomStockAnalysis').getOrCreate()

# Create SQL context object
sqlCtx = SQLContext(spark.sparkContext)

# Read in the data using the SQL context object
df = sqlCtx.read.csv('saf_stock.csv', header=True, inferSchema=True)


In [7]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [8]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [9]:
#Show the first 5 rows
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [10]:
#Use the describe method to learn about the data frame
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [12]:
# Import preparation modules
import pyspark.sql.functions as F
from pyspark.sql.functions import lit,when,col,expr,round

# Create a new data frame and round off the columns to two decimal places while adding the new HV column

df_prepared=(
df.withColumn('HV',expr("High/Volume"))#Create the new column HV which is a ratio if High to Volume ratio of stocks traded\
    .withColumn('Open', F.format_number('Open', 2))# Round the Open Column to two decimal places\
    .withColumn('High', F.format_number('High', 2))# Round the high column to two decimal places\
    .withColumn('Low', F.format_number('Low', 2))# Round the Low column to two decimal places\
    .withColumn('Close', F.format_number('Close', 2))# Round the close column to two decimal places\
    .withColumn('Volume', round('Volume', 2))#round the volume column to two decimal places.Round function used in this case to solve comma issues\
    .withColumn('Adj Close', F.format_number('Adj Close', 2))# Round the close column to two decimal places\
    .withColumn('HV',F.format_number('HV', 10)))# Round the new HV column to 10 decimal places
df_prepared.show(5)

+----------+-----+-----+-----+-----+--------+---------+------------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|          HV|
+----------+-----+-----+-----+-----+--------+---------+------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|    52.62|0.0000048197|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|    52.08|0.0000062908|
|2012-01-05|59.35|59.62|58.37|59.42|12768200|    51.83|0.0000046694|
|2012-01-06|59.42|59.45|58.87|59.00| 8069400|    51.46|0.0000073673|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|    51.62|0.0000089156|
+----------+-----+-----+-----+-----+--------+---------+------------+
only showing top 5 rows



In [13]:
#Register a table in SQL
table = df_prepared.registerTempTable("Data_Analytics")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [14]:
#Confirm that SAF has been registered
tables = sqlCtx.tableNames()

print(tables)

['data_analytics']


In [15]:
#Question 1: What day had the peak high in price?
q = "SELECT \
         Date,max(High) AS Peak_High_Price \
     FROM Data_Analytics GROUP BY Date \
    ORDER BY Peak_High_Price DESC LIMIT 1 "
sqlCtx.sql(q).show()

+----------+---------------+
|      Date|Peak_High_Price|
+----------+---------------+
|2015-01-13|          90.97|
+----------+---------------+



In [16]:
#Question 2: What is the mean of the close month?
q = "\
SELECT\
    MEAN(Close) AS MEAN\
        FROM Data_Analytics"

sqlCtx.sql(q).show()

+-----------------+
|             MEAN|
+-----------------+
|72.38844992050863|
+-----------------+



In [17]:
# What is the max and min of the volume column
q = "SELECT\
     MIN(Volume) Min_Volume,MAX(Volume) Max_Volume\
         FROM Data_Analytics\
 "

sqlCtx.sql(q).show()

+----------+----------+
|Min_Volume|Max_Volume|
+----------+----------+
|   2094900|  80898100|
+----------+----------+



In [18]:
# How many days was the close lower than 60 dollars?
q = "SELECT\
    COUNT(Date)\
    FROM Data_Analytics\
    WHERE Close <= 60\
    "
sqlCtx.sql(q).show()

+-----------+
|count(Date)|
+-----------+
|        116|
+-----------+



In [19]:
#What percentage of the time was the High greater than 80 Dollars
# Total column entries were computed earlier. An improved querry of this is to use Common Table Expressions for this querry

q = "SELECT\
     ROUND((COUNT(High)/1258*100),2) Percentage_Greater_Than_80\
         FROM Data_Analytics\
         WHERE High >= 80\
            "

sqlCtx.sql(q).show()

+--------------------------+
|Percentage_Greater_Than_80|
+--------------------------+
|                      9.14|
+--------------------------+



In [20]:
# What is the Pearson correlation between High and Volume

q = "SELECT ROUND(corr(High,Volume),2) Pearson_Correlation\
          FROM Data_Analytics"

sqlCtx.sql(q).show()

+-------------------+
|Pearson_Correlation|
+-------------------+
|              -0.34|
+-------------------+



In [21]:
#What is the Max High per Year

q = "SELECT\
     EXTRACT(YEAR FROM Date) Year,\
     MAX(High) Max_High\
     FROM Data_Analytics\
     GROUP BY Year\
     ORDER BY Max_High DESC"
sqlCtx.sql(q).show()

+----+--------+
|Year|Max_High|
+----+--------+
|2015|   90.97|
|2014|   88.09|
|2013|   81.37|
|2012|   77.60|
|2016|   75.19|
+----+--------+



In [22]:
#What is the average Close for each Calendar Month?
q = "SELECT\
    EXTRACT(MONTH FROM Date) Month,\
    ROUND(AVG(Close),2) Avg_Close\
    FROM Data_Analytics\
    GROUP BY Month\
    ORDER BY Month ASC"

sqlCtx.sql(q).show()

+-----+---------+
|Month|Avg_Close|
+-----+---------+
|    1|    71.45|
|    2|    71.31|
|    3|    71.78|
|    4|    72.97|
|    5|    72.31|
|    6|     72.5|
|    7|    74.44|
|    8|    73.03|
|    9|    72.18|
|   10|    71.58|
|   11|    72.11|
|   12|    72.85|
+-----+---------+



In [23]:
# What is the Pearson correlation between High and Volume

q = "SELECT ROUND(corr(High,Volume),2) Pearson_Correlation\
          FROM Data_Analytics"

sqlCtx.sql(q).show()

+-------------------+
|Pearson_Correlation|
+-------------------+
|              -0.34|
+-------------------+



In [24]:
#What is the Max High per Year
q = "SELECT\
     EXTRACT(YEAR FROM Date) Year,\
     MAX(High) Max_High\
     FROM Data_Analytics\
     GROUP BY Year\
     ORDER BY Max_High DESC"
sqlCtx.sql(q).show()

+----+--------+
|Year|Max_High|
+----+--------+
|2015|   90.97|
|2014|   88.09|
|2013|   81.37|
|2012|   77.60|
|2016|   75.19|
+----+--------+



In [25]:
#What is the average Close for each Calendar Month?
q = "SELECT\
    EXTRACT(MONTH FROM Date) Month,\
    ROUND(AVG(Close),2) Avg_Close\
    FROM Data_Analytics\
    GROUP BY Month\
    ORDER BY Month ASC"

sqlCtx.sql(q).show()

+-----+---------+
|Month|Avg_Close|
+-----+---------+
|    1|    71.45|
|    2|    71.31|
|    3|    71.78|
|    4|    72.97|
|    5|    72.31|
|    6|     72.5|
|    7|    74.44|
|    8|    73.03|
|    9|    72.18|
|   10|    71.58|
|   11|    72.11|
|   12|    72.85|
+-----+---------+

